In [1]:
#load packages for NLP
import spacy
nlp = spacy.load('en')

from gensim.models import Phrases
from gensim.corpora import Dictionary, MmCorpus
from gensim.models.word2vec import LineSentence
from gensim.models.ldamulticore import LdaMulticore

import pandas as pd
import numpy as np

In [2]:
#load trained n-gram and LDA models
trigram_dictionary = Dictionary.load('./models/trigram_dict_all_neg.dict')
lda = LdaMulticore.load('./models/lda_model_neg')
bigram_model = Phrases.load('./models/bigram_model_neg.txt')
trigram_model = Phrases.load('./models/trigram_model_neg.txt')

def punct_space(token):
    """
    helper function to eliminate tokens
    that are pure punctuation or whitespace
    """
    return token.is_punct or token.is_space or token.like_num or token.is_digit


def lda_descr_chat(review_text, min_topic_freq=0):
    """
    accept the original text of a review and (1) parse it with spaCy,
    (2) apply text pre-proccessing steps, (3) create a bag-of-words
    representation, (4) create an LDA representation, and
    (5) print a sorted list of the top topics in the LDA representation
    """    
    # parse the review text with spaCy
    parsed_review = nlp(review_text)
    
    # lemmatize the text and remove punctuation and whitespace
    unigram_review = [token.lemma_ for token in parsed_review
                      if not punct_space(token)]
    
    # apply the first-order and secord-order phrase models
    bigram_review = bigram_model[unigram_review]
    trigram_review = trigram_model[bigram_review]
    
    # remove any remaining stopwords
    trigram_review = [term for term in trigram_review
                      if not term in spacy.en.English.Defaults.stop_words]
    
    # create a bag-of-words representation
    review_bow = trigram_dictionary.doc2bow(trigram_review)
    
    # create an LDA representation
    review_lda = lda[review_bow]
    return max(review_lda, key=lambda item: item[1])[0]

In [3]:
#use the JSON module to parse the JSON responses from Telegram into Python dictionaries
import json 
#make web requests using Python and use it to interact with the Telegram API 
import requests 
#handle special characters
import urllib 
import time

In [4]:
#set up global variables
TOKEN = "439831904:AAGOD7SWbHzGCPq0GHxiZ0qDXocZa4HGEvc"
URL = "https://api.telegram.org/bot{}/".format(TOKEN)

In [ ]:
#download the content from a URL and giving us a string
def get_url(url):
    response = requests.get(url)
    content = response.content.decode("utf8")
    return content


#use the JSON module to parse the JSON responses from Telegram into Python dictionaries
def get_json_from_url(url):
    content = get_url(url)
    js = json.loads(content)
    return js


#retrieve a list of "updates" (messages sent to YelpReviews_bot)
def get_updates(offset=None):
    # use Long Polling; keeps the connection open until there are updates
    # in URLs, we specify that the argument list is starting with a ?
    url = URL + "getUpdates?timeout=100"
    # don't want to receive any messages with smaller IDs than this
    if offset:
        # in URLs, further arguments are separated with &
        url += "&offset={}".format(offset)
    js = get_json_from_url(url)
    return js


def get_last_update_id(updates):
    update_ids = []
    for update in updates["result"]:
        update_ids.append(int(update["update_id"]))
    return max(update_ids)


def complaint_response(updates):
    for update in updates["result"]:
        try:
            text = update["message"]["text"]
            chat = update["message"]["chat"]["id"]
            send_message(text, chat)
        except Exception as e:
            pass            

        
def get_last_chat_id_and_text(updates):
    num_updates = len(updates["result"])
    last_update = num_updates - 1
    text = updates["result"][last_update]["message"]["text"]
    chat_id = updates["result"][last_update]["message"]["chat"]["id"]
    return (text, chat_id)


#take the text of the message, and the chat ID of the chat where we want to send the message
greeting_words = ("hello", "hi", "hey", "greetings", "sup", "what's up", "/start")

def send_message(text, chat_id):
    if text.lower() in greeting_words:
        answer = "Hello, I'm the Complainteller. How can I help you?"
    else:
        complaint_type = lda_descr_chat(text)
        if complaint_type == 0:
            answer = "We're sorry you were disappointed with your meal. If you have any suggestion, please let us know."
        elif complaint_type == 1:
            answer = "We're sorry to hear that you were disappointed with our service and pricing. Unfortunately the costs are necessary to provide our quality ingredients. Thanks for your understanding."
        else:
            answer = "Thank you for bringing this situation to our attention. We would like to sincerely apologize for the disappointing customer service. Providing primary customer service is our primary goal and we will use this situation as an educational oppourtunity for our staff."
    url = URL + "sendMessage?text={}&chat_id={}".format(answer, chat_id)
    get_url(url)


def main():
    last_update_id = None
    # no duplicate messages
    while True:
        # to check 'get_updates' works
        #print("getting updates")
        updates = get_updates(last_update_id)
        #print(updates) check the current state and from/to info
        if len(updates["result"]) > 0:
            last_update_id = get_last_update_id(updates) + 1
            complaint_response(updates)
        # gets the most recent messages from Telegram every half second
        time.sleep(0.5) 


        
#import the functions into another script without running anything
if __name__ == '__main__':
    main()

/anaconda/lib/python3.6/site-packages/gensim-2.3.0-py3.6-macosx-10.7-x86_64.egg/gensim/models/phrases.py:316: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
